# Day 21: Springdroid Adventure

https://adventofcode.com/2019/day/21#part2

## Part 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from AOC2019 import Intcode, IntcodeV2, readIntcodeProg

In [2]:
prog = readIntcodeProg("./data/input21.txt")

In [3]:
def runSpringcode(prog,commands):
    # convert commands to ASCII
    com_ascii = []
    for l in commands:
        for c in l:
            com_ascii.append(ord(c))
        com_ascii.append(10)
    # initialize and run Incode
    d = IntcodeV2(prog,"DROID",com_ascii)
    d.reset()
    d.runProgram()
    #convert output to ASCII
    o = d.getOutput()
    output = []
    line = ""
    hulldamage = -1
    for i in o:
        if (i>127):
            hulldamage = i
            print("Found hull damage = ",hulldamage)
            return output, hulldamage
        if (i!=10):
            line += chr(i)
        else:
            line += '\n'
            output.append(line)
            line = ""
    return output, hulldamage

In [4]:
commands = [
    'NOT D J',
    'WALK']

# this is the first solution found by my brute force search:
#commands = ['NOT A T', 'NOT C J', 'AND D J', 'OR T J', 'WALK']

# ending commands for more clever search...
#commands = ['NOT A T', 'OR T J', 'AND D J', 'WALK' ]

# result of more clever search for Part 1
#commands = ['NOT C J', 'NOT A T', 'OR T J', 'AND D J' ] 

output, hulldamage = runSpringcode(prog,commands)

print("Hull damage =", hulldamage,"\n")
for l in output:
    print(l,end="")

Hull damage = -1 

Input instructions:

Walking...


Didn't make it across:

.................
.................
@................
#####.###########

.................
.................
.@...............
#####.###########

.................
..@..............
.................
#####.###########

...@.............
.................
.................
#####.###########

.................
....@............
.................
#####.###########

.................
.................
.....@...........
#####.###########

.................
.................
.................
#####@###########



I can attempt to brute force the solution by feeding the program all possible permutations of instructions. Of course the longer the successfull program is, the longer it'll take to find it... A solution is found with 4 instructions, but it takes quite some time!

In [5]:
# Any instruction can be composed as [AND,OR,NOT] x [A,B,C,D,T,J] x [T,J]

#bools = ["AND","OR","NOT"] # 3
bools = ["NOT","AND","OR"] # 3
tiles = ["A","B","C","D","T","J"] # 6
regs = ["T","J"] # 2

instr = []
for b in bools:
    for t in tiles:
        for r in regs:
            c = b+' '+t+' '+r
            instr.append(c)
            
#print(instr)

In [17]:
# Full brute force scan

import time
start_time = time.time()

from itertools import permutations

commands = []
output = []

found = False
#imin = 1
imin = 4
imax = 4

for n in range(imin,imax+1):
    print("Testing all possible sequences with",n,"instructions...")
    perm = permutations(instr,n)
    for p in perm:
        commands = []
        for i in p:
            commands.append(i)
        commands.append('WALK')
        output, hulldamage = runSpringcode(prog,commands)
        if (hulldamage != -1):
            print("Found solution!")
            print("Instructions =",commands)
            found = True
            break
    if found:
        break 

print("Part 1 brute force search time = %f seconds" % (time.time() - start_time))

Testing all possible sequences with 4 instructions...
Found hull damage =  19357180
Found solution!
Instructions = ['NOT A T', 'NOT C J', 'AND D J', 'OR T J', 'WALK']
Part 1 brute force search time = 5828.353593 seconds


The search space can probably be reduced by making some assumption given the result of the first tests. For instance, given the nature of a jump  (see above) the springdroid should jump when the first sensor (A) is false, and should NOT jump when the fourth sensor is false (like it is doing in the above example), so any successful program should end with these 3 instructions:

- NOT A T
- OR T J
- AND D J

This also means that the program should at least be 4 instruction long, since these 3 instructions alone fail.

In [18]:
# slightly more clever scan

endcoms = [
    'NOT A T',
    'OR T J',
    'AND D J' ]

found = False
imin = 4
imax = 10

for n in range(imin,imax+1):
    print("Testing all possible sequences with",n,"instructions...")
    perm = permutations(instr,n-len(endcoms))
    for p in perm:
        commands = []
        for i in p:
            commands.append(i)
        for i in endcoms: # add ending command sequence
            commands.append(i)
        commands.append('WALK')
        ##print(commands)
        output, hulldamage = runSpringcode(prog,commands)
        if (hulldamage != -1):
            print("Found solution! Instructions =",commands)
            found = True
            break
    if found:
        break

Testing all possible sequences with 4 instructions...
Found hull damage =  19357180
Found solution! Instructions = ['NOT C J', 'NOT A T', 'OR T J', 'AND D J', 'WALK']


## Part 2

Brute force is clearly not an option!

These are the hole patterns I should get trough:

`#####.###########`

`#####...#########`

`#####.#.#########`

`#####.#..########`

`#####.##.##.#.###`

`#####.####.#..###`

Notes:
- Jump lands 4 steps forward
- Jump if there's an hole in 3rd position (C) checkinf that 4th tile (D) is not an hole for safe landing ) solves first 2 patterns
   - `NOT C J` 
   - `AND D J`
- Adding this sequence to jump if there's an hole in front:
   - `NOT A T`
   - `OR T J`
  also solves the 4th pattern!
- For the last pattern (2 "islands") I need to jump 2 tiles before the first hole: so basically jump whenever C (3rd tile ahead) is a hole, but also checking sencond landing spot 8 tiles away (H):
  - `NOT C J` 
  - `AND D J` 
  - `AND H J`
- This fails the 6th pattern above, that I did not see before, on the second jump. I should make the second jump before, checking whether the 2nd tile (B) is an hole, similar to the sequence I added to check an hole in front before:
  - `NOT B T` 
  - `AND D T` 
  - `OR T J`

In [23]:
commands = [
    'NOT C J', 
    'AND D J',
    'AND H J',
#
    'NOT B T', 
    'AND D T', 
    'OR T J',
#
    'NOT A T',
    'OR T J',
    'RUN'
]

output, hulldamage = runSpringcode(prog,commands)

print("Hull damage =", hulldamage,"\n")
for l in output:
    print(l,end="")

Found hull damage =  1139793906
Hull damage = 1139793906 

Input instructions:

Running...

